In [ ]:
!pip install open3d==0.16.0

In [ ]:
import os
try:
    from google.colab import drive

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    print("We're running Colab")
    # Mount the Google Drive at mount
    mount = '/content/gdrive'
    print("Colab: mounting Google drive on ", mount)
    drive.mount(mount)

    # Switch to the directory on the Google Drive that you want to use
    drive_root = mount + "/My Drive/ComputerVision/"

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
        print("\nColab: making sure ", drive_root, " exists.")
        os.makedirs(drive_root, exist_ok=True)

We're running Colab
Colab: mounting Google drive on  /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).

Colab: making sure  /content/gdrive/My Drive/ComputerVision/  exists.


##Create point cloud with open3d 



In [ ]:
import cv2
import copy
import numpy as np
import open3d as o3d
import matplotlib.image as mpimg

voxel_size = 0.01

In [ ]:
cv_file = cv2.FileStorage(drive_root + "final/data/intrinsic.xml", cv2.FILE_STORAGE_READ)
intrinsics = cv_file.getNode("Camera_MatrixL").mat()
cv_file.release()

In [ ]:
camera_intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(width=306, height=408, fx=intrinsics[0][0],fy=intrinsics[1][1], cx=intrinsics[0][2], cy=intrinsics[1][2])

#from disparity to pcd (DL disp)

In [ ]:
def create_pcd_from_disp (path, intrinsic, voxel_size):
    y = mpimg.imread(path)
    yy = o3d.geometry.Image(y)

    pcd = o3d.geometry.PointCloud.create_from_depth_image(yy, intrinsic)
    pcd.transform([[-1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

    pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    return pcd

In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_plotly([source_temp, target_temp])

def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [ ]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

In [ ]:
source_path = drive_root+'final/maps/d1.png'
source = create_pcd_from_disp(source_path, camera_intrinsic_o3d, voxel_size)
o3d.visualization.draw_plotly([source])

In [ ]:
target_path = drive_root+'final/maps/d2.png'
target = create_pcd_from_disp(target_path, camera_intrinsic_o3d, voxel_size)
o3d.visualization.draw_plotly([target])

##GLOBAL REGISTRATION TO GET A GOOD INITIAL ALIGNMENT
FAST GLOBAL REGISTRATION

##Remove outliers

In [ ]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
   # o3d.visualization.draw_plotly([inlier_cloud, outlier_cloud])
    return inlier_cloud

def remove_outliers(pcd, voxel_size): 
    R = pcd.get_rotation_matrix_from_xyz((np.pi, 0, 0))
    pcd.rotate(R, center=pcd.get_center())

    print("Downsample the point cloud with a voxel of 0.02")
    voxel_down_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
        
    print("Radius oulier removal")
    cl, ind = voxel_down_pcd.remove_radius_outlier(nb_points=12, radius=0.05)
    disp_clean = display_inlier_outlier(voxel_down_pcd, ind)
    disp_clean.paint_uniform_color([0, 0, 1])
    return disp_clean

In [ ]:
clean_source = remove_outliers(source, voxel_size)
clean_target = remove_outliers(target, voxel_size)

Downsample the point cloud with a voxel of 0.02
Radius oulier removal
Showing outliers (red) and inliers (gray): 
Downsample the point cloud with a voxel of 0.02
Radius oulier removal
Showing outliers (red) and inliers (gray): 


In [ ]:
trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
                            [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
threshold = voxel_size * 0.8

In [ ]:
def registration (source, target, trans_init):
    clean_source.transform(trans_init)
    #draw_registration_result(clean_source, clean_target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(clean_source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(clean_target, voxel_size)

    result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
    print(result_fast)
    return result_fast.transformation

def registration_icp(source, target,trans_init):
    reg_p2p = o3d.pipelines.registration.registration_icp(
    clean_source, clean_target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(), o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))

    draw_registration_result(clean_source, clean_target, reg_p2p.transformation)
    target_to_source = reg_p2p.transformation
    print(reg_p2p)
    return target_to_source

In [ ]:
s = copy.deepcopy(clean_source)
t = copy.deepcopy(clean_target)

In [ ]:
trans1_2 = registration(clean_source, clean_target, trans_init)
t1_2 = registration_icp(clean_source, clean_target, trans1_2)

:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Apply fast global registration with distance threshold 0.005
RegistrationResult with fitness=9.599395e-02, inlier_rmse=3.742136e-03, and correspondence_set size of 127
Access transformation to get result.


RegistrationResult with fitness=7.396845e-01, inlier_rmse=5.136595e-03, and correspondence_set size of 1219
Access transformation to get result.


##Try to register 3 successive point clouds

#First method: create a new point cloud which is the concatenation of the two registered before and register the third with the new pcd 

In [ ]:
#create the new pcd as concatenation of the two registered before
source_temp = copy.deepcopy(clean_source)
target_temp = copy.deepcopy(clean_target)
source_temp.transform(t1_2)

first_and_second_views = source_temp + target_temp

o3d.visualization.draw_plotly([first_and_second_views])

In [ ]:
#load the third pcd
third_path = drive_root+'final/maps/d3.png'
third = create_pcd_from_disp(third_path, camera_intrinsic_o3d, voxel_size)

clean_third = remove_outliers(third, voxel_size)
th = copy.deepcopy(clean_third)

Downsample the point cloud with a voxel of 0.02
Radius oulier removal
Showing outliers (red) and inliers (gray): 


In [ ]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.7, 0.1])
    target_temp.paint_uniform_color([0.7, 0.1, 0.7])
    source_temp.transform(transformation)
    o3d.visualization.draw_plotly([source_temp, target_temp])

In [ ]:
trans1_3 = registration(first_and_second_views, clean_third, trans_init)
t1_3 = registration_icp(first_and_second_views, clean_third, trans1_3)

:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Apply fast global registration with distance threshold 0.005
RegistrationResult with fitness=8.994709e-02, inlier_rmse=3.910728e-03, and correspondence_set size of 119
Access transformation to get result.


RegistrationResult with fitness=7.463592e-01, inlier_rmse=4.978947e-03, and correspondence_set size of 1230
Access transformation to get result.


#Second method: register the second and the third and display all the three together with respective transformations

In [ ]:
trans2_3 = registration(t, th, trans_init)
t2_3 = registration_icp(t, th, trans2_3)

:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Downsample with a voxel size 0.010.
:: Estimate normal with search radius 0.020.
:: Compute FPFH feature with search radius 0.050.
:: Apply fast global registration with distance threshold 0.005
RegistrationResult with fitness=3.628118e-01, inlier_rmse=3.797315e-03, and correspondence_set size of 480
Access transformation to get result.


RegistrationResult with fitness=7.675971e-01, inlier_rmse=5.040171e-03, and correspondence_set size of 1265
Access transformation to get result.


In [ ]:
source_temp.paint_uniform_color([1, 0, 0])
target_temp.paint_uniform_color([0, 1, 0])
clean_third.paint_uniform_color([0, 0, 1])
target_temp.transform(t2_3)
o3d.visualization.draw_plotly([source_temp, target_temp, clean_third])